### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [1]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import hmac
import time
import datetime
import hashlib
import binascii

### API token, secret key and acess keys
API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [2]:
from secrets import SecretKey
from secrets import AccessKey
from novadax import RequestClient as NovaClient

nova_client = NovaClient(AccessKey, SecretKey)

### Importing tht List of currencies, making the API Call and the table Dataframe
List of crypto-currencies, get balance, and table creation

In [3]:
balance = nova_client.get_account_balance()
bdata = balance['data']

bcolumns = ['Currency', 'Balance']
bdataframe = pd.DataFrame(columns = bcolumns,)
bs = 0
bcr = {}

for bsymbol in bdata:
    pdis = bdata[bs]
    bis = pdis['balance']
    bic = pdis['currency']
    bcr[f"{bic}"] = f"{bis}"   
    bdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bcolumns),
        ignore_index = True)
    bdataframe = bdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=bcolumns),
        ignore_index=True)
    bs = bs + 1  

bdataframe.sort_values('Balance', ascending = False, inplace = True)
bdataframe.reset_index(drop = True, inplace = True)
bdataframe

,Currency,Balance
0,BTT,96.09424
1,USDT,0.09682579
2,DGB,0.0353
3,ETH,0.0138899347
4,LINK,0.0088601
5,DOGE,0.00881
6,XLM,0.00447
7,NULS,0.0044
8,IOTA,0.003905
9,XRP,0.0036


### Dolar price, timestamp and symbols 
Important cell defining symbols and the formula to the dolar price in terms of brazilian real(BRL), wich can be easily translated to other currencies or symbols.

In [4]:
symbol = 'ETH_USDT'
symbols = ['ETH_USDT', 'ETH_BRL']
currency = 'R$'
dsymbol = 'ETH_BRL'
ldolar = nova_client.list_trades(symbol, limit = 5)
my_columns = ['Symbol', 'Amount', 'Price', 'Number of Shares to Buy']
ldr_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    f = 0
    for i in ldolar['data']:
        ldolar = nova_client.list_trades(symbol, limit=10)
        ldar = ldolar['data']
        ldr = ldar[f]
        amount = ldr['amount']
        price = float(ldr['price'])
        cprice = float(amount) * price
        edolar = bcr['USDT']
        eth_wal = bcr['ETH']
        
        cwal = float(eth_wal) * price + float(edolar)
        
        sharestobuy = float(cwal/price)
                            
        
        baseVolume24h =  'N/A'
        tsp = (ldr['timestamp'])
        f = f + 1
        ldr_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        sharestobuy,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        ldr_dataframe = ldr_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        sharestobuy,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

ldr_dataframe.sort_values('Price', ascending = True, inplace = True)
ldr_dataframe.reset_index(drop = True, inplace = True)
ldr_dataframe

,Symbol,Amount,Price,Number of Shares to Buy
0,ETH_USDT,2.394,1614.57,0.013950
1,ETH_USDT,0.02,1619.82,0.013950
2,ETH_USDT,0.0337,1630.67,0.013949
3,ETH_USDT,0.2587,1640.91,0.013949
4,ETH_USDT,0.0288,1641.37,0.013949
5,ETH_BRL,0.93445871,8940.31,0.013901
6,ETH_BRL,0.13341585,8950.00,0.013901
7,ETH_BRL,0.13421839,8965.00,0.013901
8,ETH_BRL,0.11086746,8979.54,0.013901
9,ETH_BRL,0.024442,9000.00,0.013901


### Symbol position
For the execution to be done independently for each symbol, the below cell is needed:

In [5]:
my_columns = ['Symbol', 'Currency Price', 'Number of Shares to Buy']
e_dataframe = pd.DataFrame(columns = my_columns)
for symbol in symbols:
    r = 0
    for i in ldolar['data']:  
        fddata = nova_client.list_trades(symbol, limit=10)
        fdata = fddata['data']
        fdt = fdata[r]
        price = fdt['price']
        amount = fdt['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
        etsp = (fdt['timestamp'])
        r = r + 1
        for i in fddata['data'][:1]:
            e_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        shares_to_buy
                    ]
                    ,index = my_columns),
                ignore_index=True)
    
    
            e_dataframe = e_dataframe.append(
                                                pd.Series([
                                                    symbol, 
                                                    price,
                                                    shares_to_buy
                                                    ],
                                                    index = my_columns
                                                    ),
                                                ignore_index = True)

e_dataframe

,Symbol,Currency Price,Number of Shares to Buy
0,ETH_USDT,1630.67,0.013949
1,ETH_USDT,1619.82,0.013950
2,ETH_USDT,1614.57,0.013950
3,ETH_USDT,1640.91,0.013949
4,ETH_USDT,1641.37,0.013949
5,ETH_USDT,1635.6,0.013949
6,ETH_USDT,1641.75,0.013949
7,ETH_USDT,1642.09,0.013949
8,ETH_USDT,1619.87,0.013950
9,ETH_USDT,1615.59,0.013950


### Create the momentum table

### Sending the requisition to the web
The content in the order requisition should be sent to the server following the instructions given by the company or individual that holds the coin. Here as columns the last used titles shall be all used in one go to give a more complete dataframe in our xlsx database.

In [6]:
#20210222#v0-9#import all the list itens from the last dataframes and send them to the xlsx file. 
#20200225#v0-9-1# the data must come from the common filled orders or the last trades from the users history, 
#                 here the second case is uded, as well from this version on, first ones used common filled orders
#orders
listtrades = nova_client.list_orders(symbol, 'FILLED', limit=5)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']
my_columns = ['Symbol', 'Ammount','Price', 'Number of Shares to Buy', 'momentum-raw-data']
m_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listtrades = nova_client.list_trades(symbol, limit=5)
    s = 0
    for i in listtrades['data']:
        ldata = listtrades['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = ldc['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        sharestobuy = (float(edolarwal)/float(price)) 
        
        
        #M = V - Vx ; M representa o indicador de momentum, V o valor de fechamento atual e V(x_diasatras) o valor anterior
        #nao é deste modo que se utiliza o simbolo x na multiplicação. Vx seriam multiplos valores de V em x dias. Para este caso, Vi
        #veja que por conotaçao convencional parece nao fazer diferença, porém, na matemática, tais convenções são necessárias e de certo modo indispensáveis no tempo t
        #h = 
        vxdata = e_dataframe['Currency Price']
        vxi = vxdata[s]
        vx = vxi
        momentum = float(price) - float(vx)
        tsp = (ldc['timestamp'])
        s = s + 1
        m_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        sharestobuy,
                        momentum,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        m_dataframe = m_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        sharestobuy,
                        momentum,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

m_dataframe.sort_values('Price', ascending = True, inplace = True)
m_dataframe.reset_index(drop = True, inplace = True)
m_dataframe

,Symbol,Ammount,Price,Number of Shares to Buy,momentum-raw-data
0,ETH_USDT,2.394,1614.57,0.013950,0.00
1,ETH_USDT,0.02,1619.82,0.013950,0.00
2,ETH_USDT,0.0337,1630.67,0.013949,0.00
3,ETH_USDT,0.2587,1640.91,0.013949,0.00
4,ETH_USDT,0.0288,1641.37,0.013949,0.00
5,ETH_BRL,0.93445871,8940.31,0.013901,7320.49
6,ETH_BRL,0.13341585,8950.00,0.013901,7309.09
7,ETH_BRL,0.13421839,8965.00,0.013901,7350.43
8,ETH_BRL,0.11086746,8979.54,0.013901,7338.17
9,ETH_BRL,0.07543428,9040.98,0.013901,7410.31


In [7]:
#sort price [ok]
#wallet_amount [ok]
#dataframe momentum[ok]
#buy lower(wallet_amount - amount)     
#sell + profit(constant*market momentum)
#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}
#in the if statement one more contidional is needed at the "or" statment. If or else(momentum>=x or else)

### Format and export xlsx file
In the last section of the tbot a xlsx file is saved with the order table for later reading purposes.

In [8]:
writer = pd.ExcelWriter('orders.xlsx', engine = 'xlsxwriter')
m_dataframe.to_excel(writer, sheet_name = 'orderlist', index = False)

In [9]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [10]:
column_formats = {
    'A':['Symbol', string_template],
    'B':['Amount', float_template],
    'C':['Price', dollar_template],
}

for column in column_formats.keys():
    writer.sheets['orderlist'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['orderlist'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

In [11]:
writer.save()